In [2]:
from train import *
from seq2seq import *
from Encoder_search import *
from Decoder_search import *
from datasets import  load_from_disk

In [3]:
#hyperparameters
batch_size = 80       # Number of sequences in a mini-batch
vocab_size = 30000     # Size of the input vocabulary
hidden_size = 1000     # Number of features in the hidden state
embedding_dim = 620   # Word embedding dimension
context_size = ...   # Number of features in the context vector

# Getting the vocabs

In [23]:
fr_vocab_path = '../../30k_fr.txt'  # Path to the French vocab file
with open(fr_vocab_path, 'r') as file:
    fr_words = [line.strip() for line in file]

# Cleaning words and creating the dictionary
word_dict_fr = {word: i for i, word in enumerate(fr_words)}



      

eng_vocab_path= '../../30k_eng.txt'  # Path to the English vocab file
# Open the file and read lines into a list
with open(eng_vocab_path, 'r') as file:
    eng_words = [line.strip() for line in file]

word_dict_eng = { word.strip() : i for i, word in enumerate(eng_words)}

In [27]:
fr_vocab_path = '../../30k_fr.txt'  # Path to the French vocab file
with open(fr_vocab_path, 'r') as file:
    fr_words = [line.strip() for line in file]

# Creating the dictionary with the index of the first occurrence
word_dict_fr = {}
for i, word in enumerate(fr_words):
    if word not in word_dict_fr:
        word_dict_fr[word] = i



In [28]:
print(len(word_dict_eng))
print(len(word_dict_fr))
print(word_dict_fr)

30000
26684
{'PAD': 0, 'je': 1, 'de': 2, 'est': 3, 'pas': 4, 'la': 5, 'tu': 6, 'le': 7, 'vous': 8, 'ne': 9, 'il': 10, 'et': 11, 'à': 12, 'un': 13, 'ça': 14, "n'": 15, 'les': 16, 'on': 17, "l'": 18, 'une': 19, "d'": 20, 'pour': 21, 'a': 22, 'des': 23, 'en': 24, 'qui': 26, 'ce': 27, 'mais': 28, 'me': 29, 'ai': 30, 'nous': 31, 'dans': 32, 'elle': 33, 'du': 34, 'y': 35, "t'": 36, 'bien': 37, 'plus': 38, 'non': 39, 'te': 40, 'mon': 41, 'au': 42, 'avec': 43, 'moi': 44, 'va': 46, 'que': 47, 'oui': 51, 'ils': 52, 'se': 53, 'fait': 54, 'faire': 56, 'sur': 57, 'toi': 58, 'sais': 59, 'veux': 60, "s'": 62, 'ici': 63, 'rien': 64, 'était': 65, 'ma': 66, 'comme': 67, 'lui': 68, 'dit': 69, 'as': 70, 'si': 71, 'là': 72, 'cette': 73, 'votre': 74, 'es': 75, 'quand': 76, 'suis': 77, 'alors': 78, 'par': 79, 'peux': 80, 'son': 81, 'ton': 82, 'vais': 83, 'très': 84, 'ou': 85, 'dire': 86, 'quoi': 87, 'où': 88, 'sont': 89, 'allez': 90, 'aussi': 91, 'voir': 92, 'fais': 93, 'tout': 95, 'jamais': 96, 'pourquoi': 

In [5]:
print(word_dict_eng['as'])

23


# Initializing the model

In [ ]:
econder=Encoder(vocab_size,embedding_dim,hidden_size)
decoder=Decoder(vocab_size,embedding_dim,context_size,hidden_size)
model=Seq2Seq(econder,decoder)

In [ ]:
def Init_weights(self):
        for name, param in m.named_parameters():

            if 'weight_hh' in name:
                init.orthogonal_(param.data) 
            elif 'weight' in name:  
                nn.init.normal_(param.data, mean=0, std=0.01)  
            else:
            nn.init.constant_(param.data,0)

In [ ]:
data = load_from_disk('C:\\Users\\ferie\\OneDrive\\Bureau\\M2 ISI\\mini_dataset')
batches = train.make_batches(data, batch_size)